In [1]:
import warnings 
warnings.simplefilter('ignore')

import scanpy as sc
import symphonypy as sp
import os
import scparadise
import pandas as pd

In [2]:
# Load normalized integrated data
adata = sc.read_h5ad('Mouse_aging_brain/adata_unintegrated.h5ad')

In [3]:
# Create list of samples to leave in reference train dataset
lst_reference = ['old1', 'oldex1', 'oldex2', 'young2'] 

In [4]:
# Create adata_train - 8 samples of 8 donors from unintegrated mdata object (8 donors, 24 samples)
adata_train = adata[adata.obs['orig.ident'].isin(lst_reference)].copy()
# Subset anndata object based on a selected marker genes
genes = pd.read_csv('Mouse_aging_brain/genes_for_AI.csv')
adata_train = adata_train[:, genes.genes].copy()

In [5]:
# Harmony integration
adata_train.var['highly_variable'] = True
sc.pp.scale(adata_train, max_value=10) 
sc.pp.pca(adata_train, 
          use_highly_variable = True)
sp.pp.harmony_integrate(adata_train, 
                        key = "orig.ident", 
                        verbose = True, 
                        max_iter_harmony = 20)

2025-02-05 01:35:22,284 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...


Harmony integration with harmonypy is preforming.


2025-02-05 01:35:26,225 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-02-05 01:35:26,281 - harmonypy - INFO - Iteration 1 of 20
2025-02-05 01:35:28,802 - harmonypy - INFO - Iteration 2 of 20
2025-02-05 01:35:31,362 - harmonypy - INFO - Iteration 3 of 20
2025-02-05 01:35:34,098 - harmonypy - INFO - Iteration 4 of 20
2025-02-05 01:35:35,470 - harmonypy - INFO - Iteration 5 of 20
2025-02-05 01:35:36,362 - harmonypy - INFO - Iteration 6 of 20
2025-02-05 01:35:37,256 - harmonypy - INFO - Iteration 7 of 20
2025-02-05 01:35:38,206 - harmonypy - INFO - Iteration 8 of 20
2025-02-05 01:35:39,158 - harmonypy - INFO - Iteration 9 of 20
2025-02-05 01:35:40,094 - harmonypy - INFO - Converged after 9 iterations


In [6]:
# Create lists with paired samples and annotation levels
lst_test = ['young4', 'old2', 'old4', 'young1', 'oldex4']
lst_annotations = ['Celltype']
lst_predictions = ['pred_Celltype']

In [7]:
for folder in lst_test:
    os.makedirs(os.path.join('Mouse_aging_brain/symphonypy', folder))

In [8]:
for folder in lst_test:
    # Create adata_test - 2 samples of 2 donors from integrated adata object (8 donors, 24 samples)
    adata_test = adata[adata.obs['orig.ident'].isin([folder])].copy()
    adata_test.var['highly_variable'] = True
    # Mapping Harmony coordinates
    sp.tl.map_embedding(adata_query = adata_test,
                        adata_ref = adata_train)
    # Cell types prediction
    sp.tl.transfer_labels_kNN(
        adata_query = adata_test,
        adata_ref = adata_train,
        ref_labels = lst_annotations,
        query_labels = lst_predictions
    )
    # Create and save classification report of selected annotation level
    for i, j in zip(lst_annotations, lst_predictions):
        file_save = 'report_test_symphonypy_' + 'celltype_l1' + '.csv'
        scparadise.scnoah.report_classif_full(adata_test, 
                                              celltype = i, 
                                              pred_celltype = j, 
                                              report_name = file_save,
                                              save_path = os.path.join('Mouse_aging_brain/symphonypy', folder),
                                              save_report=True)

Successfully saved report

Successfully saved report

Successfully saved report

Successfully saved report

Successfully saved report



In [9]:
pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
adjustText                1.3.0
aiobotocore               2.5.4
aiohappyeyeballs          2.4.2
aiohttp                   3.8.4
aioitertools              0.12.0
aiosignal                 1.3.1
airr                      1.5.1
alembic                   1.13.3
anndata                   0.9.1
annoy                     1.17.3
anyio                     3.6.2
appdirs                   1.4.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
array_api_compat          1.8
arrow                     1.2.3
asciitree                 0.3.3
asttokens                 2.4.1
async-lru                 2.0.4
async-timeout             4.0.2
attrs                     23.1.0
awkward                   2.7.1
awkward_cpp               42
babel                     2.16.0
bamnostic                 1.1.10
bbknn                     1.6.0
bcrypt                    4.2.1
beautifulsoup4            